# 1. Introduction and Schema Diagram

In this guided project, we're going to practice using our SQL skills to answer business questions.

We'll continue to use the Chinook database that we've used in earlier lessons of this course. The Chinook database is provided as a SQLite database file called chinook.db. A copy of the database schema is below - you'll need to come back to this step often to consult the schema as you write your queries.

![](https://s3.amazonaws.com/dq-content/191/chinook-schema.svg)

In [8]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

# 2. Overview of the Data

In [9]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [10]:
%%sql

SELECT * FROM employee LIMIT 5;

 * sqlite:///chinook.db
Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
1,Adams,Andrew,General Manager,None,1962-02-18 00:00:00,2016-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
2,Edwards,Nancy,Sales Manager,1,1958-12-08 00:00:00,2016-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
4,Park,Margaret,Sales Support Agent,2,1947-09-19 00:00:00,2017-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
5,Johnson,Steve,Sales Support Agent,2,1965-03-03 00:00:00,2017-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com


In [6]:
%%sql

SELECT * FROM customer LIMIT 5;

 * sqlite:///chinook.db
Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


# 3. Selecting Albums to Purchase

In [13]:
%%sql

WITH tot_count AS
                    (
                        SELECT COUNT(*) AS total_tracks
                            FROM tracks
                    )

SELECT g.name AS genre, COUNT(t.name), t.country AS genre_number
    FROM genre AS g
INNER JOIN track AS t ON t.genre_id=g.genre_id
INNER JOIN invoice AS i on i.invoice:id
GROUP BY g.name

 * sqlite:///chinook.db
(sqlite3.OperationalError) no such column: t.country
[SQL: WITH tot_count AS (
                        SELECT COUNT(*) AS total_tracks
                            FROM tracks
                    )

SELECT g.name AS genre, COUNT(t.name), t.country AS genre_number
    FROM genre AS g
INNER JOIN track AS t ON t.genre_id=g.genre_id
GROUP BY g.name]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


In [42]:
%%sql

WITH usa_tracks_sold AS
                        (
                            SELECT 
                                il.*
                            FROM invoice_line AS il
                            INNER JOIN invoice AS i on i.invoice_id=il.invoice_id
                            INNER JOIN customer AS c on c.customer_id=i.customer_id
                        WHERE c.country="USA"
                        )
    
SELECT 
        g.name AS genre,
        COUNT(uts.invoice_id) AS tracks_sold,
        CAST(COUNT(uts.invoice_id) AS Float)/
            (SELECT COUNT(*) FROM usa_tracks_sold) AS percentage_sold
FROM usa_tracks_sold AS uts
INNER JOIN track AS t on t.track_id=uts.track_id
INNER JOIN genre AS g on g.genre_id=t.genre_id
GROUP BY genre
ORDER BY tracks_sold DESC

 * sqlite:///chinook.db
Done.


genre,tracks_sold,percentage_sold
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Pop,22,0.02093244529019981
Latin,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


We can see that Rock music has by far the most track sales out of any genre in the United States. They make up a majority of the sales.

After that is Alt/Punk and Metal at around 12% each.

# 4. Analyzing Employee Sales Performance

In [58]:
%%sql

SELECT DISTINCT (first_name || " " || last_name), title FROM employee

 * sqlite:///chinook.db
Done.


"(first_name || "" "" || last_name)",title
Andrew Adams,General Manager
Nancy Edwards,Sales Manager
Jane Peacock,Sales Support Agent
Margaret Park,Sales Support Agent
Steve Johnson,Sales Support Agent
Michael Mitchell,IT Manager
Robert King,IT Staff
Laura Callahan,IT Staff


In [68]:
%%sql

SELECT (e.first_name || " " || e.last_name) AS employee_name,
        ROUND(SUM(il.unit_price), 2) AS total_sales,
        e.hire_date,
        (e.address || " " ||e.city || ", " || e.state || " " || e.country  )
    FROM employee AS e
INNER JOIN customer AS c ON c.support_rep_id = e.employee_id
INNER JOIN invoice AS i ON i.customer_id=c.customer_id
INNER JOIN invoice_line AS il ON il.invoice_id=i.invoice_id
GROUP BY employee_name

 * sqlite:///chinook.db
Done.


employee_name,total_sales,hire_date,"(e.address || "" "" ||e.city || "", "" || e.state || "" "" || e.country )"
Jane Peacock,1731.51,2017-04-01 00:00:00,"1111 6 Ave SW Calgary, AB Canada"
Margaret Park,1584.0,2017-05-03 00:00:00,"683 10 Street SW Calgary, AB Canada"
Steve Johnson,1393.92,2017-10-17 00:00:00,"7727B 41 Ave Calgary, AB Canada"
